In [1]:
import pandas as pd
import requests
import config

In [4]:
response_list = []
API_KEY = config.api_key

for movie_id in range(550,556): 
  url = 'https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id, API_KEY)
  r = requests.get(url)
  response_list.append(r.json())

In [5]:
df = pd.DataFrame.from_dict(response_list)

In [6]:
df

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,/rr7E0NoGKxvbkb89eR1GwfoYjpA.jpg,None,63000000,"[{'id': 18, 'name': 'Drama'}]",http://www.foxmovies.com/movies/fight-club,550,tt0137523,en,Fight Club,...,1999-10-15,100853753,139,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Mischief. Mayhem. Soap.,Fight Club,False,8.4,24336
1,False,/v1QEIuBM1vvpvfqalahhIyXY0Cm.jpg,"{'id': 372257, 'name': 'The Poseidon Adventure...",5000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",,551,tt0069113,en,The Poseidon Adventure,...,1972-12-13,84563118,117,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,"Hell, upside down.",The Poseidon Adventure,False,7.1,657
2,False,/k4JIHyAXaGHwAwT7y5Skd17f0Wl.jpg,None,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",,552,tt0237539,it,Pane e tulipani,...,2000-03-03,8478434,114,"[{'english_name': 'Italian', 'iso_639_1': 'it'...",Released,Imagine your life. Now go live it.,Bread and Tulips,False,7.3,209
3,False,/r3xsFBD1VTUusk393bBc7SsDUJe.jpg,"{'id': 1952, 'name': 'USA: Land of Opportuniti...",10000000,"[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...",,553,tt0276919,en,Dogville,...,2003-05-19,16680836,178,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,A quiet little town not far from here.,Dogville,False,7.8,1947
4,False,/1qwXItFKqvKYyW1CwbYhxyUC8Pj.jpg,None,0,"[{'id': 18, 'name': 'Drama'}, {'id': 36, 'name...",,554,tt0308476,ru,Кукушка,...,2002-01-01,0,100,"[{'english_name': 'German', 'iso_639_1': 'de',...",Released,She's Making Peace One Man at a Time.,The Cuckoo,False,7.1,65
5,False,None,None,0,"[{'id': 53, 'name': 'Thriller'}]",http://www.luecke-im-system.de/,555,tt0442896,en,Absolut,...,2005-04-20,0,94,"[{'english_name': 'French', 'iso_639_1': 'fr',...",Released,,Absolut,False,7.9,20


In [7]:
df_columns = ['budget', 'genres', 'id', 'imdb_id', 'original_title', 'release_date', 'revenue', 'runtime']

In [10]:
df['genres']

0                        [{'id': 18, 'name': 'Drama'}]
1    [{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...
2    [{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...
3    [{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...
4    [{'id': 18, 'name': 'Drama'}, {'id': 36, 'name...
5                     [{'id': 53, 'name': 'Thriller'}]
Name: genres, dtype: object

In [11]:
genres_list = df['genres'].tolist()
flat_list = [item for sublist in genres_list for item in sublist]

In [12]:
result = []
for l in genres_list:
    r = []
    for d in l:
        r.append(d['name'])
    result.append(r)
df = df.assign(genres_all=result)

In [13]:
df_genres = pd.DataFrame.from_records(flat_list).drop_duplicates()
df_genres

,id,name
0,18,Drama
1,28,Action
2,12,Adventure
3,53,Thriller
4,35,Comedy
5,10749,Romance
6,80,Crime
10,36,History


In [14]:
df_columns = ['budget', 'id', 'imdb_id', 'original_title', 'release_date', 'revenue', 'runtime']
df_genre_columns = df_genres['name'].to_list()
df_columns.extend(df_genre_columns)

s = df['genres_all'].explode()
df = df.join(pd.crosstab(s.index, s))

In [15]:
df['release_date'] = pd.to_datetime(df['release_date'])
df['day'] = df['release_date'].dt.day
df['month'] = df['release_date'].dt.month
df['year'] = df['release_date'].dt.year
df['day_of_week'] = df['release_date'].dt.day_name()
df_time_columns = ['id', 'release_date', 'day', 'month', 'year', 'day_of_week']

In [16]:
df[df_columns].to_csv('tmdb_movies.csv', index=False)
df_genres.to_csv('tmdb_genres.csv', index=False)
df[df_time_columns].to_csv('tmdb_datetimes.csv', index=False)